# Testing ⏭ GreedLR Scheduler for 🤗 LLMs

### Need to reinstall from source to register changes

(may need to restart kernel)

In [2]:
# %pip install -r translation/requirements.txt
%pip install -e ~/transformers/ #Or wherever you downloaded this source 

Obtaining file:///root/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.9 MB/s eta 0:00:0000:01
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.27.0.dev1-0.editable-py3-none-any.whl size=34553 sha256=e7019d9047978c3a76949176c22868da797855a387ed8d7f53b4167d52cc268f
  Stored in directory: /tmp/pip-ephem-wheel-cache-903koijc/wheels/cc/64/f7/a67713e0143d17a61a8c81af64dffa96f04d6602a4e4d50e71
Successfully built transformers

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] T

In [5]:
%pip install -U datasets peft accelerate trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 1.9 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from transformers import AutoModelForCausalLM, Trainer, AutoTokenizer, GreedyLR, TrainingArguments
from datasets import load_dataset
from peft import PeftModel, LoraConfig

### Load dataset

In [15]:

from datasets import load_dataset

dataset = load_dataset("imdb", split="train")
dataset["text"][100]

"Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary."

### Load model

In [29]:
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
import torch


peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    # gradient_accumulation_steps=4,
    gradient_checkpointing =False,
    max_grad_norm= 0.3,
    num_train_epochs=30, 
    learning_rate=1e-3,
    bf16=False,
    save_total_limit=3,
    logging_steps=10,
    output_dir='./greedylr_llm_results/',
    optim="adamw_hf",
    lr_scheduler_type="greedy",
    warmup_ratio=0.05, #for cosine
    factor=0.9 # for greedylr
)

trainer = SFTTrainer(
    "EleutherAI/gpt-neo-125m",
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    args=training_args
)

Using pad_token, but it is not set yet.


### Train!

In [30]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


GreedyLR settings: patience=10 smooth=False min_lr=0.001 factor=0.9


Step,Training Loss
10,3.656200
20,3.709100
30,3.709100
40,3.724800
50,3.671300
60,3.664200
70,3.734400
80,3.681800
90,3.671300
100,3.617900


KeyboardInterrupt: 